<a href="https://colab.research.google.com/github/MarianaPerciani/CinemAI/blob/main/projeto_alura_gemini_cinemai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
###############################################
# --- Agente 1: Agente Validador Resposta --- #
###############################################

def agente_validador_resposta_booleano(pergunta, resposta):
    buscador = Agent(
        name="agente_validador_resposta",
        model=MODEL_ID,
        instruction=f"""
        Você é um assistente validador de resposta de usuário.
        A pergunta feita foi: "{pergunta}".
        A resposta do usuário foi: "{resposta}".
        Sua tarefa é verificar se a resposta do usuário é válida e relevante para a pergunta.
        Se a resposta for válida, retorne a palavra exata: VERDADEIRO.
        Se a resposta não for válida ou não fizer sentido, retorne a palavra exata: FALSO.
        """,
        description="Agente validador de resposta (retorna booleano).",
    )

    entrada_do_agente_validador_resposta = f"Pergunta: {pergunta}\nResposta: {resposta}"
    resposta_do_agente_validador_resposta = call_agent(buscador, entrada_do_agente_validador_resposta)

    # Processa a resposta do agente para retornar um booleano Python
    if "VERDADEIRO" in resposta_do_agente_validador_resposta:
        return True
    else:
        return False

In [9]:
##################################################
# --- Agente 2: Agente de Feedback Elaborado --- #
##################################################

def agente_feedback_elaborado(pergunta, resposta):
    agente_feedback = Agent(
        name="agente_feedback_elaborado",
        model=MODEL_ID,
        instruction=f"""
        Você é um assistente de feedback de respostas dados por um usuário para preenchimento de um formulário.
        A pergunta feita ao usuário foi: "{pergunta}".
        A resposta do usuário foi: "{resposta}".
        Sua tarefa é analisar a resposta do usuário e fornecer um feedback breve, amigável e útil,
        indicando por que a resposta pode não estar correta e, se possível, dando uma instrução
        ou sugestão para ajudar o usuário a responder corretamente, mas sem sugerir uma resposta.
        Use um tom amigável e instruitivo com, no máximo, 150 caracteres.
        """,
        description="Agente para gerar feedback elaborado sobre respostas inválidas.",
    )

    entrada_do_agente_feedback = f"Pergunta: {pergunta}\nResposta: {resposta}"
    resposta_do_agente_feedback = call_agent(agente_feedback, entrada_do_agente_feedback)
    return resposta_do_agente_feedback

def obter_resposta_validada_com_feedback(pergunta):
    while True:
        resposta_usuario = input(f"{pergunta}\nSua resposta: ")
        valida = agente_validador_resposta_booleano(pergunta, resposta_usuario)
        if valida:
            print("Resposta processada.")
            return resposta_usuario
        else:
            feedback = agente_feedback_elaborado(pergunta, resposta_usuario)
            print(feedback)
            print("Por favor, tente responder novamente.")

In [10]:
##################################################
# --- Agente 3: Buscador de Cinemas Próximos --- #
##################################################

def agente_busca_cinema(endereco):
    buscador_cinema = Agent(
        name="agente_busca_cinema",
        model=MODEL_ID,
        instruction=f"""
        Você é um especialista em encontrar cinemas próximos a um determinado endereço.
        O endereço do usuário é: "{endereco}".
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar os 10 cinemas mais próximos deste endereço.
        Na sua resposta, liste o nome de cada cinema e sua respectiva distância do endereço do usuário.
        """,
        description="Agente que busca os 10 cinemas mais próximos de um endereço.",
        tools=[google_search]
    )

    entrada_do_agente_busca_cinema = f"Endereço do usuário: {endereco}"
    # Executa o agente
    resultado_busca_cinema = call_agent(buscador_cinema, entrada_do_agente_busca_cinema)
    return resultado_busca_cinema

In [11]:
#######################################################
# --- Agente 4: Buscador de Programação Detalhada --- #
#######################################################

def agente_busca_programacao(lista_de_cinemas, data_cinema):
    buscador_programacao = Agent(
        name="agente_busca_programacao_detalhada",
        model=MODEL_ID,
        instruction=f"""
        Você é um especialista em encontrar a programação detalhada de cinemas.
        Você recebeu a seguinte lista de cinemas:
        {lista_de_cinemas}
        E a data desejada para a programação é: "{data_cinema}".

        Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar a programação completa de filmes para cada um dos cinemas listados na data especificada.

        Para cada cinema, liste os filmes em exibição e, para cada filme, tente encontrar e listar os seguintes detalhes:
        - Horários das sessões para a data "{data_cinema}".
        - Se a sessão é dublada (DUB) ou legendada (LEG).

        Organize a resposta de forma clara, listando cada cinema, os filmes em exibição naquele cinema e os detalhes de cada sessão (horário e áudio).
        """,
        description="Agente que busca a programação detalhada (horário e áudio) dos cinemas para uma data específica.",
        tools=[google_search]
    )

    entrada_do_agente_busca_programacao = f"Lista de Cinemas: {lista_de_cinemas}\nData do Cinema: {data_cinema}"
    # Executa o agente
    resultado_busca_programacao = call_agent(buscador_programacao, entrada_do_agente_busca_programacao)
    return resultado_busca_programacao


In [12]:
###########################################
# --- Agente 5: Filtro de Programação --- #
###########################################

def agente_filtra_programacao(programacao, preferencia_audio, periodo_preferido):
    filtro_programacao = Agent(
        name="agente_filtra_programacao",
        model=MODEL_ID,
        instruction=f"""
        Você é um especialista em filtrar a programação de cinemas com base nas preferências do usuário.

        Você recebeu a seguinte programação completa dos cinemas para a data "{data_cinema}":
        {programacao}

        As preferências do usuário para o filme são:
        - Preferência por áudio (dublado/legendado): "{preferencia_audio}"
        - Período do dia preferido (manhã/tarde/noite/qualquer): "{periodo_preferido}"

        Sua tarefa é analisar a programação completa e retornar apenas as opções de filmes e sessões que correspondam às preferências do usuário.

        Filtre a programação da seguinte forma:
        1. Inclua apenas as sessões que correspondam à preferência de áudio do usuário (se especificada como "dublado" ou "legendado"). Se a preferência for "qualquer", inclua todas as sessões.
        2. Inclua apenas as sessões que ocorram dentro do período do dia preferido do usuário (se especificado como "manhã", "tarde" ou "noite"). Se a preferência for "qualquer", inclua todas as sessões. Considere os seguintes horários:
           - Manhã: 06:00 - 12:00
           - Tarde: 12:00 - 18:00
           - Noite: 18:00 - 00:00

        Retorne uma nova estrutura contendo apenas os cinemas, filmes e sessões que atendem aos critérios de filtro, incluindo o horário e o tipo de áudio (DUB/LEG) de cada sessão.
        """,
        description="Agente que filtra a programação de cinemas com base nas preferências de áudio e período do dia do usuário.",
        # Não precisa de ferramentas externas para este filtro básico
    )

    entrada_do_agente_filtro = f"""
    Programação Completa: {programacao}
    Data do Cinema: {data_cinema}
    Preferência de Áudio: {preferencia_audio}
    Período Preferido: {periodo_preferido}
    """
    # Executa o agente
    programacao_filtrada = call_agent(filtro_programacao, entrada_do_agente_filtro)
    return programacao_filtrada

In [13]:
#################################################
# --- Agente 6: Seletor de Melhores Sessões --- #
#################################################

def agente_seleciona_melhor_sessao_final(programacao_filtrada, genero_preferido, acompanhantes):
    seletor_sessoes_final = Agent(
        name="agente_seleciona_melhor_sessao_final",
        model=MODEL_ID,
        instruction=f"""
        Você é um especialista em recomendar as melhores sessões de cinema para um usuário,
        considerando a programação filtrada e suas preferências adicionais.

        Você recebeu a seguinte programação filtrada:
        {programacao_filtrada}

        As preferências adicionais do usuário são:
        - Gênero de filme preferido (se houver): "{genero_preferido}"
        - Irá acompanhado? (sozinho/amigos/família/encontro): "{acompanhantes}"

        Sua tarefa é analisar a programação filtrada e selecionar até 3 das melhores opções de sessões para o usuário.

        Para cada filme na programação filtrada, use a ferramenta de busca do Google (google_search) para obter:
        - Avaliações do filme: verificar nota da avaliação no Letterbox, Rotten Tomatoes e demais sites de critica.
        - Uma breve sinopse do filme.
        - O gênero do filme (para confirmar ou refinar a correspondência com a preferência do usuário).

        Priorize as sessões da seguinte forma:
        1. Filmes com as melhores avaliações (considere a média das avaliações encontradas).
        2. Filmes que correspondam ao gênero preferido do usuário (se especificado).
        3. Filmes que pareçam adequados para o tipo de companhia do usuário (interprete essa preferência com base na sinopse e gênero do filme - ex: filmes infantis para família, filmes de comédia ou ação para amigos, filmes românticos para encontro).

        Retorne uma lista de até 3 opções de sessões, formatadas da seguinte maneira para cada opção:
        - Filme: [Título do Filme]
        - Cinema: [Nome do Cinema]
        - Horário: [Horário da Sessão]
        - Áudio: [DUB/LEG]
        - Comentário: [Uma breve explicação de por que esta opção foi selecionada, considerando as preferências do usuário e as avaliações do filme, bem como descrever brevemente a sinopse.]
        """,
        description="Agente que seleciona as 3 melhores sessões de cinema com base em avaliações, gênero e companhia.",
        tools=[google_search]
    )

    entrada_do_agente_selecao_final = f"""
    Programação Filtrada: {programacao_filtrada}
    Gênero Preferido: {genero_preferido}
    Acompanhantes: {acompanhantes}
    """
    # Executa o agente
    melhores_sessoes_final = call_agent(seletor_sessoes_final, entrada_do_agente_selecao_final)
    return melhores_sessoes_final

In [14]:
# Lista de perguntas
lista_de_perguntas = [
    "Qual dia pretende ir ao cinema? Informar, preferencialmente no formato DD/MM/AAAA.",
    "Tem preferência por filmes dublados ou legendados?",
    "Você irá sozinho, com amigos, família ou encontro?",
    "Possui algum gênero de filme preferido?",
    "Tem preferência por algum período do dia?",
    "Qual seu endereço? Se possível, informar bairro e cidade.",
]

# Lista para armazenar as respostas validadas
respostas_usuario = []

# Iterando sobre a lista de perguntas e obtendo respostas validadas
for pergunta in lista_de_perguntas:
    resposta = obter_resposta_validada_com_feedback(pergunta)
    respostas_usuario.append(resposta)

data_cinema = respostas_usuario[0]
preferecia_audio = respostas_usuario[1]
acompanhantes = respostas_usuario[2]
genero = respostas_usuario[3]
preferencia_periodo = respostas_usuario[4]
endereco= respostas_usuario[5]


Qual dia pretende ir ao cinema? Informar, preferencialmente no formato DD/MM/AAAA.
Sua resposta: 18/05/2025
Resposta processada.
Tem preferência por filmes dublados ou legendados?
Sua resposta: legendado
Resposta processada.
Você irá sozinho, com amigos, família ou encontro?
Sua resposta: com minha família
Resposta processada.
Possui algum gênero de filme preferido?
Sua resposta: comédia
Resposta processada.
Tem preferência por algum período do dia?
Sua resposta: após almoço e antes de anoitecer
Resposta processada.
Qual seu endereço? Se possível, informar bairro e cidade.
Sua resposta: vila mariana, são paulo
Resposta processada.


In [15]:
cinemas_proximos = agente_busca_cinema(endereco)
#display(to_markdown(cinemas_proximos))

In [16]:
programacao = agente_busca_programacao(cinemas_proximos, data_cinema)
#display(to_markdown(programacao))

In [17]:
programacao_filtrada = agente_filtra_programacao(programacao, preferecia_audio, preferencia_periodo)
#display(to_markdown(programacao_filtrada))

In [18]:
sessoes_selecionadas = agente_seleciona_melhor_sessao_final(programacao_filtrada, genero, acompanhantes)
display(to_markdown(sessoes_selecionadas))

> Okay, com base na programação filtrada e nas suas preferências (comédia para assistir com a família), vou procurar as melhores opções de filmes.
> 
> 
> Com base nas informações que encontrei, aqui estão as 3 melhores opções de sessões para você e sua família, considerando que você prefere comédia e um filme adequado para assistir em família:
> 
> *   **Filme:** Detetive Chinatown: O Mistério de 1900
> *   **Cinema:** Cinemark Shopping Metrô Santa Cruz
> *   **Horário:** 16:50
> *   **Áudio:** LEG
> *   **Comentário:** Este filme é uma comédia de mistério que pode agradar a família toda. A sinopse indica uma história envolvente com toques de humor, e a classificação etária (12 anos) sugere que é adequado para crianças mais velhas e adolescentes. As avaliações indicam uma média de 3.5/5, o que sugere que é um filme divertido, embora não seja um clássico.
> 
> *   **Filme:** Karatê Kid Lendas
> *   **Cinema:** Cinemark Shopping Metrô Santa Cruz
> *   **Horário:** 17:40
> *   **Áudio:** LEG
> *   **Comentário:** Este filme parece ser uma opção nostálgica e revigorante para toda a família. A história de um jovem prodígio do kung fu que se muda para Nova York pode atrair tanto crianças quanto adultos. As críticas destacam que não é necessário conhecer os filmes anteriores ou a série Cobra Kai para aproveitar este filme, tornando-o acessível para todos. Além disso, a combinação de Jackie Chan e Ralph Macchio no elenco pode trazer um toque especial para os fãs dos filmes originais.
> 
> *   **Filme:** Pequenas Coisas Como Estas
> *   **Cinema:** Cinema Reserva Cultural São Paulo
> *   **Horário:** 17:10
> *   **Áudio:** LEG
> *   **Comentário:** Este filme pode ser uma boa escolha se você procura algo um pouco mais profundo e emocionante para assistir com a família. A história de Bill Furlong, um vendedor de carvão que faz uma descoberta perturbadora, pode gerar discussões importantes sobre valores e consciência. Embora não seja uma comédia, pode ser uma experiência enriquecedora para compartilhar com a família.
